In [50]:
np.load("/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/SS03112022_02/used_init_probe.npy").shape

(1, 2561, 2561)

In [ ]:
import numpy as np
from sscPimega import pi540D
from sscCdi import read_hdf5, Geometry

input_dict = {}
input_dict["temporary_output"] = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/SS03112022_02/temp"
input_dict["GPUs"] = [0,1,2]
input_dict["DP_center"] = [1418,1383]
input_dict["detector_ROI_radius"] = 1280
input_dict["flatfield"] = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/data/ptycho2d/SS03112022_02/images/flat.hdf5"
input_dict["mask"] = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/data/ptycho2d/SS03112022_02/images/mask.hdf5"
input_dict["detector_distance"] = 14.000035595703125
input_dict["suspect_border_pixels"] = 3 
input_dict["fill_blanks"] = False

geometry, params = Geometry(input_dict["detector_distance"]*1000,susp=input_dict["suspect_border_pixels"],fill=input_dict["fill_blanks"]) # distance in milimeters

dic = {}
# dic['path']     = ["/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/data/ptycho2d/SS03112022_02/scans/0000_SS03112022_02_001.hdf5"]
dic['path'] = ["/home/ABTLUS/yuri.tonin/test/ex.py"]
dic['outpath']  = "/home/ABTLUS/yuri.tonin/test/" #input_dict["temporary_output"]
dic['order']    = "yx" 
dic['rank']     = "ztyx" # order of axis
dic['dataset']  = "entry/data/data"
dic['gpus']     = input_dict["GPUs"]
dic['init']     = 0
dic['final']    = -1 # -1 to use all DPs
dic['saving']   = 1  # save or not
dic['timing']   = 0  # print timers 
dic['blocksize']= 10
dic['center']   = (input_dict["DP_center"][1],input_dict["DP_center"][0]) # [1400,1400]
dic['roi'] = input_dict["detector_ROI_radius"] # integer
if 1: 
    print(dic)
    dic['flat'] = np.ones([3072, 3072]) 
    dic['mask'] = np.zeros([3072, 3072])
else:
    dic['flat'] = read_hdf5(input_dict["flatfield"])[()][0, 0, :, :] # np.ones([3072, 3072]) #
    dic['mask'] = read_hdf5(input_dict["mask"])[()][0, 0, :, :] # np.zeros([3072, 3072])
dic['empty']    = np.zeros_like(dic['flat']) # OBSOLETE! empty is not used anymore;        dic['daxpy']    = [0,np.zeros([3072,3072])] 
dic['daxpy']    = [0,np.zeros([3072,3072])] 
dic['geometry'] = geometry

dic['path'] = dic['path'][0]
restored_data_info = pi540D.ioSet_Backward540D( dic )
    
DPs = pi540D.ioGetM_Backward540D( dic, restored_data_info, 0)


{'path': ['/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/data/ptycho2d/SS03112022_02/scans/0000_SS03112022_02_001.hdf5'], 'outpath': '/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/SS03112022_02/temp', 'order': 'yx', 'rank': 'ztyx', 'dataset': 'entry/data/data', 'gpus': [0, 1], 'init': 0, 'final': -1, 'saving': 1, 'timing': 0, 'blocksize': 10, 'center': (1383, 1418), 'roi': 1280}


In [3]:
import h5py
path = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/data/ptycho2d/SS03112022_02/images/mask.hdf5"
file = h5py.File(path,'r')
mask = file['entry/data/data'][0][0]

In [ ]:
path_empty = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/data/ptycho2d/SS03112022_02/images/empty.hdf5"
file = h5py.File(path_empty,'r')
empty = np.asarray(file['entry/data/data'])[0][0]
empty.shape

(3072, 3072)

In [42]:
path_flat = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/data/ptycho2d/SS03112022_02/images/flat.hdf5"
file = h5py.File(path_flat,'r')
flat = np.asarray(file['entry/data/data'])[0][0]
flat.shape

False

In [4]:
path = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/SS03112022_02/2023-04-12-11h31m_SS03112022_02.hdf5"
file = h5py.File(path,'r')
DP = np.asarray(file['log/raw_DP_avg'])
print(DP.shape)
DP2 = DP.copy()

(3072, 3072)


In [ ]:
path_data = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/data/ptycho2d/SS03112022_02/scans/0000_SS03112022_02_001.hdf5"
file = h5py.File(path_data,'r')
data = np.asarray(file['entry/data/data'])[0]
print(data.shape)

(1, 3072, 3072)


In [ ]:
path1 = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/SS03112022_02/used_DPs.npy"
path2 = "/ibira/lnls/beamlines/caterete/apps/gcc-jupyter/00000000/proc/recons/SS03112022_02/best/used_DPs.npy"

import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
from matplotlib.colors import LogNorm
from sscPimega import opt540D, pi540D
new = np.load(path1)
old = np.load(path2)
print(new.shape, old.shape)
N = 622
for i in range(new.shape[0]):
    # print(i, np.max(new[i]),np.min(new[i]),np.mean(new[i]), new.shape)
    if np.max(new[i]) == 0: print(i)

new = new[N]
old = old[N]
print(new.shape,old.shape)
fig, ax = plt.subplots(1,2,dpi=300)
ax0, ax1= ax.ravel()
# ax0.imshow(new,norm=LogNorm())

new2 = new.copy()
diff = new2 - old
center = 1280
L = 50
# diff = diff[center-L:center+L,center-L:center+L]
new = new[center-L:center+L,center-L:center+L]
old = old[center-L:center+L,center-L:center+L]
# ax0.imshow(diff)
ax0.imshow(new)#,norm=LogNorm())
ax1.imshow(old)#,norm=LogNorm())
# ax2.imshow(new_mean,norm=LogNorm())
# ax3.imshow(old_mean,norm=LogNorm())

In [48]:
l = [8,18,28,38,48,58,68,78,88,98,108,118,128,138,148,158,168,178,188,198,208,218,228,238,248,258,268,278,288,298,308,318,328,338,348,358,368,378,388,398,408,418,428,438,448,458,468,478,488,498,508,518,528,538,548,558,568,578,588,598,608,618,619,620,621,622,623]

In [50]:
3072-1383, 3072-1418

(1689, 1654)